In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.model_selection import train_test_split
import json

import glob
import random
import collections
from glob import glob
import numpy as np
import pandas as pd

#import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

from skimage.io import imread
from skimage.transform import resize
from keras import layers
from keras import models
import keras
from keras.layers import LeakyReLU
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import shutil
import keras
from PIL import Image
#import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
import json
import tensorflow as tf 
from keras.layers import Input
from keras import Sequential
from keras.layers import Dense, LSTM,Flatten, TimeDistributed, Conv2D, Dropout
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import re
import string

from keras.models import load_model
from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K
import numpy as np
import pandas as pd

from skimage.io import imread
from skimage.transform import resize
from keras import layers
from keras import models
import keras
from keras.layers import LeakyReLU
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Conv1D,Reshape, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten, UpSampling2D
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import pandas as pd

from skimage.io import imread
from skimage.transform import resize
from keras import layers
from keras import models
import keras
from keras.layers import LeakyReLU
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K

import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.layers import Lambda, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras import Input
from keras.layers import TimeDistributed, Conv2D, Dense, MaxPooling2D, Flatten, LSTM, Dropout, BatchNormalization
from keras import models
AUTO = tf.data.AUTOTUNE
from sklearn.metrics import confusion_matrix
import random

AUTO = tf.data.AUTOTUNE
BATCH_SIZE = 32
EPOCHS = 50
CROP_TO = 32
SEED = 26

PROJECT_DIM = 128
LATENT_DIM = 512
WEIGHT_DECAY = 0.0005
learning_rate = 0.0001
batch_size = 128
hidden_units = 512
projection_units = 256
num_epochs = 2
dropout_rate = 0.5

temperature = 0.05

In [ ]:
!pip install tensorflow-addons

In [2]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [3]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
#train_labels = to_categorical(train_labels)
train_images = train_images.astype('float32')
test_images = test_images.astype('float32')
train_images = train_images / 255
test_images = test_images / 255 

x_train, x_unlab, y_train, y_unlab = train_test_split( train_images, train_labels , test_size=0.5, random_state=42 )
X_train, X_test, Y_train, Y_test = train_test_split( x_train,y_train , test_size=0.2, random_state=40 )

In [17]:
len(x_train)

50000

In [4]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.Normalization(),
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.02),
        layers.experimental.preprocessing.RandomWidth(0.2),
        layers.experimental.preprocessing.RandomHeight(0.2),
    ]
)

# Setting the state of the normalization layer.
data_augmentation.layers[0].adapt(X_train)

In [5]:


num_classes = 10
input_shape = (32, 32, 3)

def create_encoder():
    resnet = tf.keras.applications.ResNet50V2( include_top=False, weights="imagenet", input_shape=input_shape, pooling="avg" )

    inputs = keras.Input(shape=input_shape)
    augmented = data_augmentation(inputs)
    outputs = resnet(augmented)
    model = keras.Model(inputs=inputs, outputs=outputs, name="cifar10-encoder")
    return model

encoder = create_encoder()
encoder.summary()


Model: "cifar10-encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 sequential (Sequential)     (None, 32, 32, 3)         7         
                                                                 
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
Total params: 23,564,807
Trainable params: 23,519,360
Non-trainable params: 45,447
_________________________________________________________________


In [6]:
def create_classifier(encoder, trainable=True):

    for layer in encoder.layers:
        layer.trainable = trainable

    inputs = keras.Input(shape=input_shape)
    features = encoder(inputs)
    features = layers.Dropout(dropout_rate)(features)
    features = layers.Dense(hidden_units, activation="relu")(features)
    features = layers.Dropout(dropout_rate)(features)
    outputs = layers.Dense(num_classes, activation="softmax")(features)

    model = keras.Model(inputs=inputs, outputs=outputs, name="cifar10-classifier")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )
    return model

In [7]:
class SupervisedContrastiveLoss(keras.losses.Loss):
    def __init__(self, temperature=1, name=None):
        super(SupervisedContrastiveLoss, self).__init__(name=name)
        self.temperature = temperature

    def __call__(self, labels, feature_vectors, sample_weight=None):

        feature_vectors_normalized = tf.math.l2_normalize(feature_vectors, axis=1)

        logits = tf.divide( tf.matmul(  
            feature_vectors_normalized, tf.transpose(feature_vectors_normalized)),self.temperature,)
        
        return tfa.losses.npairs_loss(tf.squeeze(labels), logits)


def add_projection_head(encoder):
    
    inputs = keras.Input(shape=input_shape)
    features = encoder(inputs)
    outputs = layers.Dense(projection_units, activation="relu")(features)
    model = keras.Model(
        inputs=inputs, outputs=outputs, name="cifar-encoder_with_projection-head"
    )
    
    return model

In [8]:
encoder = create_encoder()

encoder_with_projection_head = add_projection_head(encoder)

encoder_with_projection_head.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=SupervisedContrastiveLoss(temperature),
)

encoder_with_projection_head.summary()

history = encoder_with_projection_head.fit(
    x=X_train, y=Y_train, batch_size=batch_size, epochs=2)

Model: "cifar-encoder_with_projection-head"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 cifar10-encoder (Functional  (None, 2048)             23564807  
 )                                                               
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
Total params: 24,089,351
Trainable params: 24,043,904
Non-trainable params: 45,447
_________________________________________________________________
Epoch 1/2
157/157 [==============================] - 51s 171ms/step - loss: 6.2643
Epoch 2/2
157/157 [==============================] - 18s 115ms/step - loss: 4.6970


In [12]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [13]:
tf.test.is_gpu_available()

True

In [9]:
classifier = create_classifier(encoder, trainable=False) 

history = classifier.fit(x=X_train, y=Y_train, batch_size=batch_size, epochs=num_epochs) 

accuracy = classifier.evaluate(X_test, Y_test)[1] 
print(f"Test accuracy: {round(accuracy * 100, 2)}%") 

Epoch 1/2
157/157 [==============================] - 16s 45ms/step - loss: 2.6423 - sparse_categorical_accuracy: 0.1870
Epoch 2/2
157/157 [==============================] - 10s 47ms/step - loss: 1.7564 - sparse_categorical_accuracy: 0.3674
Test accuracy: 36.74%


In [10]:
# Helper function for merging new history objects with older ones
def append_history(losses, val_losses, accuracy, val_accuracy, history):
    losses = losses + history.history["loss"]
    val_losses = val_losses + history.history["val_loss"]
    accuracy = accuracy + history.history["accuracy"]
    val_accuracy = val_accuracy + history.history["val_accuracy"]
    return losses, val_losses, accuracy, val_accuracy


# Plotter function
def plot_history(losses, val_losses, accuracies, val_accuracies):
    plt.plot(losses)
    plt.plot(val_losses)
    plt.legend(["train_loss", "val_loss"])
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.show()

    plt.plot(accuracies)
    plt.plot(val_accuracies)
    plt.legend(["train_accuracy", "val_accuracy"])
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.show()

In [ ]:


def resmodel():
    
    
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), input_shape=(32, 32, 3)))
    model.add(layers.Conv2D(32, (3, 3), activation="relu"))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3),))
    model.add(layers.Conv2D(64, (3, 3),))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3),))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(.2))
    model.add(layers.Flatten()) 
    model.add(layers.Dense(32))

    model.summary()
    return model

In [11]:
from keras.layers import TimeDistributed, Conv2D, Dense, MaxPooling2D, Flatten, LSTM, Dropout, BatchNormalization
from keras import models
AUTO = tf.data.AUTOTUNE

def create_model(X,Y,X_test, Y_test,num_epochs):
    
    data_augmentation.layers[0].adapt(X)
    
    encoder = create_encoder()

    encoder_with_projection_head = add_projection_head(encoder)
    
    encoder_with_projection_head.compile(optimizer=keras.optimizers.Adam(learning_rate),
                                     loss=SupervisedContrastiveLoss(temperature),)
    
    
    history = encoder_with_projection_head.fit(x=X, y=Y, batch_size=256, epochs=num_epochs)
    
    
    classifier = create_classifier(encoder, trainable=False) 

    history = classifier.fit(x=X, y=Y, batch_size=batch_size, epochs=num_epochs) 

    accuracy = classifier.evaluate(X_test, Y_test)[1] 
    
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    #model.summary()
    return encoder,classifier

In [12]:
arr = np.concatenate((X_train, x_unlab))
X_all = arr
arr = np.concatenate((Y_train, y_unlab))
Y_all = arr

In [13]:
# callbacks=[keras.callbacks.EarlyStopping(patience=4, verbose=1), ],

def train_full_model(X_train, X_test, Y_train, Y_test,num):
    
    X_train, Y_train = shuffle(X_train, Y_train)
    
    encoder,classifier = create_model(X_train,Y_train,X_test, Y_test,num)

    accuracy = classifier.evaluate(X_test, Y_test)[1] 
    print(f"Test accuracy: {round(accuracy * 100, 2)}%") 
    
    
    return encoder,classifier


encoder,classifier = train_full_model(X_all, X_test, Y_all, Y_test,1)

157/157 [==============================] - 9s 42ms/step - loss: 1.6691 - sparse_categorical_accuracy: 0.4218
Test accuracy: 42.18%
157/157 [==============================] - 5s 33ms/step - loss: 1.6691 - sparse_categorical_accuracy: 0.4218
Test accuracy: 42.18%


In [ ]:
encoder,classifier = train_full_model(X_train, X_test, Y_train, Y_test,5)

In [14]:
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

def train_active_learning_models(encoder,classifier,X_train,Y_train,x_unlab,y_unlab,X_test,Y_test,num_iterations,num_epochs=1):

    # Creating lists for storing metrics
    losses, val_losses, accuracies, val_accuracies = [], [], [], []


    try :
        accuracy = classifier.evaluate(X_test, Y_test, batch_size=32)[1]
    except :
        return encoder, classifier,X_train,Y_train,x_unlab,y_unlab 
    
    print("Acc Test : ")
    print("\n")
    print(accuracy)
    print("\n")
   
    
    
    l = len(y_unlab)
    d = int ( np.round ( l/num_iterations ) )
    #x = int(np.round( l/d ))
    
    for iteration in range(10):
        
        l = len(y_unlab)
        
        print("\n\n\n\n")
        
        print("Iteration : ")
        
        print(iteration+1)
        
        print("\n\n\n\n\n")

  
        try :
            x_ulb = encoder.predict(x_unlab, batch_size=batch_size)
        except :
            return encoder, classifier,X_train,Y_train,x_unlab,y_unlab 
        
        print(np.shape(x_ulb))


        nn_clusters = 10
        num_points_per_class = int ( np.round (d/10)   )
        
        budget =  50
        num_points_per_class = 50
        
        print("\n")
        print("Annotated in each iter : ")
        print(budget*10)
        print("\n")
        print("Chosen in each iter : ")
        print(num_points_per_class*10)
        print("\n")

    
        kmeans = KMeans(n_clusters=nn_clusters, init='k-means++', n_init=10).fit(x_ulb)

        closest_points_indices = []
        annotate_indices = []
        
        for i in range(10):
            
            cluster_center = kmeans.cluster_centers_[i]
            
            cluster_indices = np.where(kmeans.labels_ == i)[0]
            
            distances = np.linalg.norm(x_ulb[cluster_indices] - cluster_center, axis=1)
            
            closest_indices = cluster_indices[np.argsort(distances)[:budget]]
            
            annotate = cluster_indices[np.argsort(distances,-1)[:budget]]
            
            closest_points_indices.extend(closest_indices)
            
            annotate_indices.extend(annotate)

        chosen_indices = closest_points_indices
    
        print(np.shape(chosen_indices))
        print(np.shape(annotate_indices))
        
        rnd = chosen_indices
        
        annt = annotate_indices
        
        
        all = list(range(1, l))
        main_list = list(set(all) - set(rnd) - set(annt))
        
        #add those index to from unlablled set to training set
        new_lab = x_unlab[rnd]
        new_annt = x_unlab[annt]
        arr = np.concatenate((X_train, new_lab,new_annt))
        X_train = arr

        
        try :
            accuracy = classifier.evaluate(new_lab, y_unlab[rnd], batch_size=64)[1]
        except :
            return encoder, classifier,X_train,Y_train,x_unlab,y_unlab 
        
        print("Acc : ")
        print("\n")
        print(accuracy)
        
        #predict on the set and add to training data
        try :
            new_y = np.round(classifier.predict(new_lab, batch_size=64))
        except :
            return encoder, classifier,X_train,Y_train,x_unlab,y_unlab 
        annt_y = y_unlab[annt]
        print("This")
        print(np.shape(Y_train))
        print("Now")
        print(np.shape(new_y))
        
        new_yy = np.argmax(new_y, axis=1)
        
        new_yy = new_yy.reshape(len(new_y),1)
        
        
        arr = np.concatenate((Y_train, new_yy,annt_y))
        Y_train = arr
        
        
        #create the new unlabelled set
        x_unlab = x_unlab[main_list]

        #create the new unlabelled label set
        y_unlab = y_unlab[main_list]
        
        
        try :
            history = encoder_with_projection_head.fit(x=X_train, y=Y_train, batch_size=batch_size, epochs=5)
        except :
            return encoder, classifier,X_train,Y_train,x_unlab,y_unlab 
        
        print("Now2")
        print(np.shape(Y_train))
        print("\n")

        try :
            history = classifier.fit(x=X_train, y=Y_train, batch_size=64, epochs=5) 
        except :
            return encoder, classifier,X_train,Y_train,x_unlab,y_unlab 
        
        #test on data
    
    try :
        accuracy = classifier.evaluate(X_test, Y_test, batch_size=8)[1]
    except :
        return encoder, classifier,X_train,Y_train,x_unlab,y_unlab 
    
    print("Acc Test : ")
    print("\n")
    print(accuracy)
    print("\n")
   
    
    return encoder, classifier,X_train,Y_train,x_unlab,y_unlab 

from tensorflow.keras.applications import ResNet50V2
import numpy as np
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

import numpy as np
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Load CIFAR-10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train_flat = y_train.flatten()

# Set random seed for reproducibility
np.random.seed(42)

# Choose 5 random classes to undersample
all_classes = np.arange(10)
low_classes = np.random.choice(all_classes, 5, replace=False)
high_classes = np.setdiff1d(all_classes, low_classes)

# Create imbalanced training set
x_new, y_new = [], []

for cls in all_classes:
    idx = np.where(y_train_flat == cls)[0]
    if cls in low_classes:
        selected_idx = idx[:500]
    else:
        selected_idx = idx
    x_new.append(x_train[selected_idx])
    y_new.append(y_train[selected_idx])

# Combine and shuffle the imbalanced training data
x_train = np.concatenate(x_new, axis=0)
y_train = np.concatenate(y_new, axis=0)

# Final shuffle
final_idx = np.random.permutation(len(x_train))
x_train = x_train[final_idx]
y_train = y_train[final_idx]

# Normalize and one-hot encode
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
#y_train = to_categorical(y_train, 10)
#y_test = to_categorical(y_test, 10)



x_train, x_unlab, y_train, y_unlab = train_test_split( x_train, y_train , test_size=0.4, random_state=42 )
X_train, X_test, Y_train, Y_test = train_test_split( x_train,y_train , test_size=0.5, random_state=40 )

print(len(Y_train))
print(len(Y_test))

data_augmentation.layers[0].adapt(X_train)
encoder = create_encoder()
encoder_with_projection_head = add_projection_head(encoder)
encoder_with_projection_head.compile(optimizer=keras.optimizers.Adam(learning_rate),
                        loss=SupervisedContrastiveLoss(temperature))
    
history = encoder_with_projection_head.fit(x=X_train, y=Y_train, batch_size=batch_size, epochs=10)
classifier = create_classifier(encoder, trainable=False) 
history = classifier.fit(x=X_train, y=Y_train, batch_size=64, epochs=10) 

encoder, classifier,X_train,Y_train,x_unlab,y_unlab = train_active_learning_models(encoder, classifier,X_train,Y_train,x_unlab,y_unlab,x_test,y_test,num_iterations=3)

8250
8250
Epoch 1/10
65/65 [==============================] - 24s 128ms/step - loss: 8.0198
Epoch 2/10
65/65 [==============================] - 8s 127ms/step - loss: 4.9923
Epoch 3/10
65/65 [==============================] - 7s 102ms/step - loss: 4.7582
Epoch 4/10
65/65 [==============================] - 7s 103ms/step - loss: 4.6621
Epoch 5/10
65/65 [==============================] - 7s 111ms/step - loss: 4.5721
Epoch 6/10
65/65 [==============================] - 8s 122ms/step - loss: 4.4990
Epoch 7/10
65/65 [==============================] - 8s 124ms/step - loss: 4.4396
Epoch 8/10
65/65 [==============================] - 7s 106ms/step - loss: 4.3716
Epoch 9/10
65/65 [==============================] - 7s 103ms/step - loss: 4.3125
Epoch 10/10
65/65 [==============================] - 8s 130ms/step - loss: 4.2367
Epoch 1/10
129/129 [==============================] - 18s 69ms/step - loss: 1.8925 - sparse_categorical_accuracy: 0.4444
Epoch 2/10
129/129 [==============================] - 6s 

203/203 [==============================] - 9s 42ms/step - loss: 0.4065 - sparse_categorical_accuracy: 0.8823
Epoch 3/5
203/203 [==============================] - 9s 44ms/step - loss: 0.4083 - sparse_categorical_accuracy: 0.8805
Epoch 4/5
203/203 [==============================] - 9s 46ms/step - loss: 0.4062 - sparse_categorical_accuracy: 0.8821
Epoch 5/5
203/203 [==============================] - 9s 42ms/step - loss: 0.4002 - sparse_categorical_accuracy: 0.8794





Iteration : 
6






(8625, 2048)


Annotated in each iter : 
500


Chosen in each iter : 
500


(462,)
(462,)
8/8 [==============================] - 1s 108ms/step - loss: 0.6628 - sparse_categorical_accuracy: 0.8009
Acc : 


0.8008658289909363
This
(12990, 1)
Now
(462, 10)
Epoch 1/5
109/109 [==============================] - 13s 115ms/step - loss: 3.5173
Epoch 2/5
109/109 [==============================] - 10s 94ms/step - loss: 3.5147
Epoch 3/5
109/109 [==============================] - 11s 105ms/step - loss: 3.4972
Epoch 

16:22:59.202783


In [32]:
Y_train

array([[8],
       [7],
       [8],
       ...,
       [3],
       [1],
       [3]], dtype=uint8)

In [33]:
a =np.argmax(Y_train)
a

20

In [30]:
a[1]

IndexError: index 1 is out of bounds for axis 0 with size 1